In [62]:
from imdb import IMDb
import pickle
ia = IMDb()
movie_list = []
for number in range(1, 20):
    movie_id = str(number).zfill(7)
    current_movie = ia.get_movie(movie_id)
    if len(current_movie) > 0:
        movie_list.append(current_movie)
with open("test.txt", "wb") as myfile:
    pickle.dump(movie_list, myfile)

2017-03-31 14:40:00,326 WARNING [imdbpy.parser.http.urlopener] /Users/hung-yiwu/anaconda/lib/python2.7/site-packages/IMDbPY-5.1-py2.7-macosx-10.6-x86_64.egg/imdb/parser/http/__init__.py:248: 404 code returned for //akas.imdb.com/title/tt0000000/combined: Not Found (headers: Date: Fri, 31 Mar 2017 18:40:00 GMT
Server: HTTPDaemon
X-Frame-Options: SAMEORIGIN
Cache-Control: private
Content-Type: text/html; charset=iso-8859-1
Set-Cookie: cache=BCYg6kcMVJeHy8U2dSjgnHGy0hBvUwlObY7-WzQ6d4rCVwqpcmoazx5GHlUc8kZZ1eaWlurHrlWCYhJychq91J0Rcfzqb9CpUxXG394bSgowT5Q;path=/;domain=.imdb.com
Set-Cookie: uu=BCYntWEfwyXsi160arC96FnKeBFvUwlObY7-WzQ6d4rCVwpjgjh3YjEgBvfAybUCJDXp0TdfYqHrOYoOtgMi4TAyvF9_-Xw1By224BIeJKjaXiiKdSGlf5HC0dMjRSBAVUrju-pFwU9HC4jyfRHKI1DTA7YAhZKVS8kxhHOEAG3zVAsanm_Wxvg2BR1_0mCJ71q-t_FDzAQRoSQdreSRm0xgBzgWlCJL2pmQ9kzEWb6eutJ8S8oSZdu1ywXGzGTJ_PjjARdXdNZhslhLaq1z_Z94qvt1VSyq1SRUXxkKIotPMyYQV3MhRHN_x_zQG-nuSirTmWMEAqzdPec33bObEsQOiw;expires=Thu, 30 Dec 2037 00:00:00 GMT;path=/;domain=.imdb.c

In [99]:
import httplib

conn = httplib.HTTPSConnection("api.themoviedb.org")

payload = "{}"

conn.request("GET", "/3/find/tt0000001?api_key=7fbe755e9e83c66a573c6482cefd23ed&language=en-US&external_source=imdb_id", payload)

res = conn.getresponse()
data = res.read()

print data.decode("utf-8")

{"movie_results":[{"adult":false,"backdrop_path":null,"genre_ids":[99],"id":16612,"original_language":"en","original_title":"Carmencita","overview":"The first woman to appear in front of an Edison motion picture camera and possibly the first woman to appear in a motion picture within the United States. In the film, Carmencita is recorded going through a routine she had been performing at Koster &amp; Bial's in New York since February 1890.","release_date":"1894-03-14","poster_path":"/6QJowxF4f2hrUtmrDXOqbRDMdDH.jpg","popularity":1.004496,"title":"Carmencita","video":false,"vote_average":5.2,"vote_count":13}],"person_results":[],"tv_results":[],"tv_episode_results":[],"tv_season_results":[]}
